In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Convert Low Quality Audio to 44.1 kHz
#
# CONVERTS LOW QUALITY AUDIO MP3 TO 44100 Hz
#
# This section of the code is responsible for converting low-quality audio files to a standard sample rate of 44100 Hz,
# which is required for podcast uploads. The function defined below will scan through a specified directory for MP3 files,
# check their sample rates, and convert any files with a sample rate lower than 44100 Hz to the target sample rate.
# The conversion ensures that all audio files meet the required quality standards for the podcast platform.
#
# Written by Jeremy Wheeler
#

!apt-get install -y ffmpeg
!pip install pydub tqdm

import os
from pydub import AudioSegment
from tqdm import tqdm

# Function to convert the sample rate of MP3 files in a directory to the target sample rate
def convert_sample_rate(raw_directory, pending_directory, target_sample_rate=44100):
    # Collect all MP3 files in the directory and its subdirectories
    mp3_files = []
    for root, _, files in os.walk(raw_directory):
        for file in files:
            if file.endswith('.mp3'):
                mp3_files.append(os.path.join(root, file))

    # Initialize the progress bar
    for file_path in tqdm(mp3_files, desc="Converting audio files", unit="file"):
        file = os.path.basename(file_path)
        # Load the audio file using pydub
        audio = AudioSegment.from_file(file_path)

        # Check if the audio file's sample rate is less than the target sample rate
        if audio.frame_rate < target_sample_rate:
            print(f'Converting {file} from {audio.frame_rate}Hz to {target_sample_rate}Hz...')
            # Convert the audio file's sample rate to the target sample rate
            converted_audio = audio.set_frame_rate(target_sample_rate)
            # Define the path for the converted audio file in the pending directory
            converted_file_path = os.path.join(pending_directory, file)
            # Export the converted audio file in MP3 format
            converted_audio.export(converted_file_path, format='mp3')
            print(f'Saved converted file to {converted_file_path}')
        else:
            # Skip conversion if the audio file's sample rate is already at or above the target sample rate
            print(f'Skipping {file}, sample rate is {audio.frame_rate}Hz')

# Specify the directories containing the MP3 files and where to move the converted files
mp3_directory1 = '/content/drive/MyDrive/input'
pending_directory = '/content/drive/MyDrive/output'

# Ensure the pending directory exists
os.makedirs(pending_directory, exist_ok=True)

# Call the function to convert the sample rate of MP3 files in the specified directory and move them to the pending directory
convert_sample_rate(mp3_directory1, pending_directory)
